### Read Videos & Convert to Frames

In [2]:
import os
#import logging
#import subprocess
import pandas as pd

In [3]:
#logging.basicConfig(level=logging.INFO) 

### Convert to Grayscale

In [4]:
from matplotlib import pyplot as plt
import cv2
import numpy as np

### Directional ATD

In [5]:
def performThreshold(diffFrame):
    mask = (diffFrame < 13) & (diffFrame > -13)
    diffFrame[mask] = 0
    return diffFrame

In [6]:
def binarize(noiseless_image):
    mean = np.mean(noiseless_image)
    maskZeroes = (noiseless_image < mean)
    maskOnes = (noiseless_image >= mean)
    binarizedImage = noiseless_image
    binarizedImage[maskZeroes] = 0
    binarizedImage[maskOnes] = 1
    return binarizedImage

In [7]:
def accumulateFrames(framesList):
    accumulatedFrame = framesList[0]

    for i in range(1, len(framesList)):
        accumulatedFrame += framesList[i]
    
    return accumulatedFrame

In [8]:
def applyForwardDiff(grayImagesList):
    listOfFrames = []

    for index in range(len(grayImagesList) - 1):
        forward_ImageDiff = grayImagesList[index] - grayImagesList[index + 1]
        forward_ImageDiff = performThreshold(forward_ImageDiff)
        binarizedImage = binarize(forward_ImageDiff)        
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

In [9]:
def applyBackwardDiff(grayImagesList):
    listOfFrames = []

    for index in range(1, len(grayImagesList)):
        backward_ImageDiff = grayImagesList[index] - grayImagesList[index - 1]
        backward_ImageDiff = performThreshold(backward_ImageDiff)
        binarizedImage = binarize(backward_ImageDiff) 
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

In [10]:
def applyBiDirectionalDiff(grayImagesList):
    listOfFrames = []

    for index in range(1, len(grayImagesList)-1):
        average_frame = np.mean([grayImagesList[index - 1], grayImagesList[index + 1]], axis=0)
        bidirectional_ImageDiff = grayImagesList[index] - average_frame
        bidirectional_ImageDiff = performThreshold(bidirectional_ImageDiff)
        binarizedImage = binarize(bidirectional_ImageDiff)
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

In [11]:
def convertGrayScaletoList(input_directory):
    gray_images = []

    for filename in sorted(os.listdir(input_directory)):
        if filename.endswith(".jpg"):
            image_path = os.path.join(input_directory, filename)
            gray_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            gray_images.append(gray_image)

    return gray_images

In [12]:
df = pd.read_excel("Dataset Excel Sheets/ArabSign Labeled Sentence.xlsx", header=None, names=['Key', 'Value'])
data_dict = {key: [int(i) for i in str(values).split(',') if i.strip().isdigit()] for key, values in zip(df['Key'], df['Value'])}

In [13]:
def applyATD(frames, output_directory, index, sent):
    forwardATD = applyForwardDiff(frames)
    backwardATD = applyBackwardDiff(frames)
    bidirectATD = applyBiDirectionalDiff(frames)
    
    
    wordFolder = os.path.join(output_directory, f'word_{index}')

    # check if the folder already exists
    if not os.path.exists(wordFolder):
        os.makedirs(wordFolder)
        #logging.info(f"Folder created at {wordFolder}")
    #else:
        #logging.info(f"Folder already exists at {wordFolder}")

    #ATDFolder = os.path.join(wordFolder, 'ATD')

    # check if the folder already exists
    #if not os.path.exists(ATDFolder):
    #    os.makedirs(ATDFolder)
        #logging.info(f"Folder created at {ATDFolder}")
    #else:
        #logging.info(f"Folder already exists at {ATDFolder}")

    # Save the atd images (RGB) to a directory
    #image_ForwardATD_path = os.path.join(ATDFolder, f"forwardATD_word{index}.png")
    #cv2.imwrite(image_ForwardATD_path, forwardATD)
    
    #image_BackwardATD_path = os.path.join(ATDFolder, f"backwardATD_word{index}.png")
    #cv2.imwrite(image_BackwardATD_path, backwardATD)
    
    #image_BidirectionalATD_path = os.path.join(ATDFolder, f"bidirectionalATD_word{index}.png")
    #cv2.imwrite(image_BidirectionalATD_path, bidirectATD)
        
    #logging.info(f"ATD frames '{output_directory[-10:]}_word{index}' created and saved in '{ATDFolder}'")

    stackedImage = np.dstack([forwardATD, backwardATD, bidirectATD]).astype(np.uint8)
    stackedImage_path = os.path.join(wordFolder, f"word_{index}.png")#f"{data_dict[sent+1][index]}.png")
    cv2.imwrite(stackedImage_path, stackedImage)
    
    #logging.info(f"Stacked Image '{output_directory[-10:]}_word{index}' created and saved in '{wordFolder}'")

### Main

#### _Iterating Through Files & Run:_
*   Image Sequences
*   Grayscaling
*   ATD & Image Stacking

In [14]:

dataset_path = r"D:\LINA SCHOOL LAPTOP\MODELS\ArabSign Predict Test Greyscales"                            # path to whole Dataset Folder
ATD_output_path = r"D:\LINA SCHOOL LAPTOP\MODELS\ArabSign Predict Stacks"       # path to a folder of your choice
if not os.path.exists(ATD_output_path):
    os.mkdir(ATD_output_path)

folders = [participant for participant in os.listdir(dataset_path)]                         # each 6 participents
for participant in folders:
    new_part = os.path.join(ATD_output_path, participant)
    if not os.path.exists(new_part):
        os.mkdir(new_part)
    new_test = os.path.join(new_part, 'test')
    if not os.path.exists(new_test):
        os.mkdir(new_test)                                         
    test_path = os.path.join(dataset_path, participant, 'test')                            # directly open the test for each participant
    for idx, sentence_folder in enumerate(os.listdir(test_path)):
        sentence_path = os.path.join(test_path, sentence_folder)                           # get the path to a folder for one sentence
        new_sent = os.path.join(new_test, sentence_folder)
        if not os.path.exists(new_sent):
            os.mkdir(new_sent)
        gray_folders = [os.path.join(sentence_path, item) for item in os.listdir(sentence_path) if os.path.isdir(os.path.join(sentence_path, item))]
        for gray_fold in gray_folders:
            vid_name = os.path.splitext(os.path.basename(gray_fold))[0]                       # using the same name as the video (without .mp4 as the output folder name)
            new_vid = os.path.join(new_sent, vid_name)
            if not os.path.exists(new_vid):
                os.mkdir(new_vid)                                                      # for every training video in that sentence folder
            
                #noWords = numWordsArr[idx]
                
                frames = convertGrayScaletoList(gray_fold)
                
                noFrames = len(frames)
                if noFrames <= 98:
                    noWords = 2
                elif noFrames > 98 and noFrames <= 132:
                    noWords = 3
                elif noFrames >= 133 and noFrames <= 147:
                    noWords = 4
                elif noFrames >= 148:
                    noWords = 5
                
                split = noFrames // noWords
                for n in range(noWords): 
                    index = split * n
                    if n == noWords-1:
                        applyATD(frames[index:], new_vid, n, idx)
                    else:
                        applyATD(frames[index: index + split], new_vid, n, idx)
        